<a href="https://colab.research.google.com/github/VictoriaIngham/ITN_Timecourse/blob/main/Final_visualisation_TiemCourse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import Combobox, VBox, Output
from IPython.display import display
from google.colab import files
import io

# === Upload CSV ===
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')), sep=None, engine='python')

# === Clean ID column ===
df = df[df['ID'].notna()]
df['ID'] = df['ID'].astype(str).str.strip().str.upper()
gene_ids = sorted(df['ID'].unique())

# === Setup ===
timepoints = ['1', '4', '8', '24', '48', '72']
conditions = ['CFPR', 'PYR', 'Combined']
log2fc_columns = {
    cond: [(tp, f'log2FoldChange.{cond}_vs_Control_{tp}') for tp in timepoints]
    for cond in conditions
}

# === Color map ===
condition_colors = {
    "CFPR": "#E69F00",
    "PYR": "#009E73",
    "Combined": "#CC79A7"
}

# === Widgets ===
gene_box = Combobox(
    options=gene_ids,
    placeholder='Start typing gene ID...',
    description='Gene ID:',
    ensure_option=True,
    continuous_update=True
)
output = Output()

# === Plotting function ===
def plot_expression(change=None):
    gene_id = gene_box.value
    if not gene_id:
        return

    gene_id = gene_id.strip().upper()
    gene_data = df[df['ID'] == gene_id]

    with output:
        output.clear_output()
        if gene_data.empty:
            print(f"Gene '{gene_id}' not found.")
            return

        # Plot
        plt.figure(figsize=(10, 6))
        for cond in conditions:
            x = [int(tp) for tp, _ in log2fc_columns[cond]]
            y = gene_data[[col for _, col in log2fc_columns[cond]]].values.flatten()
            sns.lineplot(x=x, y=y, marker='o', label=cond, color=condition_colors[cond])

        plt.axhline(0, linestyle='--', color='gray')
        plt.xlabel('Time (hr)')
        plt.ylabel('log2 Fold Change')
        plt.title(f'Expression of {gene_id} Over Time')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Table
        table = {
            f'{cond}_log2FC_{tp}h': gene_data[f'log2FoldChange.{cond}_vs_Control_{tp}'].values[0]
            for cond in conditions for tp in timepoints
        }
        display(pd.DataFrame([table]))

# === Link callback ===
gene_box.observe(plot_expression, names='value')

# === Display UI ===
display(VBox([gene_box, output]))


Saving final_combined_results_fixed.csv to final_combined_results_fixed.csv
